In [2]:
#!pip install scikit-learn
#!pip install pandas
#!pip install numpy

In [3]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MaxAbsScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold, cross_val_score

In [4]:
# Carregar o dataset
df = pd.read_csv('filmes_filtrados.csv')

# Converter a coluna 'genres' para uma lista de gêneros
df['genres_names'] = df['genres'].apply(lambda x: x.split('-'))

# Tratando a data
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['release_year'] = df['release_date'].dt.year
df['release_month'] = df['release_date'].dt.month

# Explodindo a coluna 'genres_names' para que cada gênero fique em uma linha separada
df = df.explode('genres_names').reset_index(drop=True)

# Retirando colunas desnecessárias para a análise
df = df.drop(columns=['id', 'title', 'status', 'release_date', 'genres'])
df.head()


,original_language,popularity,budget,revenue,runtime,vote_average,vote_count,genres_names,release_year,release_month
0,en,8763.998,129000000.0,352056482.0,116.0,7.079,1365.0,Action,2023,8
1,en,8763.998,129000000.0,352056482.0,116.0,7.079,1365.0,Science Fiction,2023,8
2,en,8763.998,129000000.0,352056482.0,116.0,7.079,1365.0,Horror,2023,8
3,en,5953.227,18000000.0,65675816.0,103.0,7.433,545.0,Horror,2023,4
4,en,5953.227,18000000.0,65675816.0,103.0,7.433,545.0,Mystery,2023,4


In [5]:
features = ['original_language', 'popularity', 'budget', 'runtime', 'vote_average', 'vote_count', 'genres_names','release_year', 'release_month']
features_nominais = ['genres_names', 'original_language']
X = df[features]
y = df['revenue']
# Dividindo o dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=87)

In [6]:
# Pipeline para colunas nominais
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), features_nominais),
        ('num', MaxAbsScaler(), [col for col in features if col not in features_nominais])
    ]
)

# Pipeline completo com KNN
pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('regressor', KNeighborsRegressor())
])


In [7]:
pipeline.fit(X_train, y_train)

,steps,"[('preprocess', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('cat', ...), ('num', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [8]:

kf = KFold(n_splits=5, shuffle=True, random_state=87)
scores = cross_val_score(pipeline, X_train, y_train, cv=kf, scoring='r2')
print("R² em cada fold:", scores)
print("R² médio:", scores.mean())

R² em cada fold: [0.66953953 0.70479653 0.70179499 0.65345265 0.67280551]
R² médio: 0.6804778393760575


In [9]:
from sklearn.preprocessing import MinMaxScaler

# Pipeline para colunas nominais com MinMaxScaler
preprocessor_minmax = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), features_nominais),
        ('num', MinMaxScaler(), [col for col in features if col not in features_nominais])
    ]
)

pipeline_minmax = Pipeline([
    ('preprocess', preprocessor_minmax),
    ('regressor', KNeighborsRegressor())
])

# Treinando o pipeline com MinMaxScaler
pipeline_minmax.fit(X_train, y_train)

# Avaliando o modelo com cross-validation
scores_minmax = cross_val_score(pipeline_minmax, X_train, y_train, cv=kf, scoring='r2')
print("R² em cada fold (MinMax):", scores_minmax)
print("R² médio (MinMax):", scores_minmax.mean())

R² em cada fold (MinMax): [0.66888988 0.7093098  0.71112324 0.65073514 0.66948083]
R² médio (MinMax): 0.6819077781456075


In [10]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

# Pipeline para colunas nominais com StandardScaler e SVM
preprocessor_standard = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), features_nominais),
        ('num', StandardScaler(), [col for col in features if col not in features_nominais])
    ]
)

pipeline_svm = Pipeline([
    ('preprocess', preprocessor_standard),
    ('regressor', SVR())
])

# Treinando o pipeline com SVM
pipeline_svm.fit(X_train, y_train)

# Avaliando o modelo com cross-validation
scores_svm = cross_val_score(pipeline_svm, X_train, y_train, cv=kf, scoring='r2')
print("R² em cada fold (SVM):", scores_svm)
print("R² médio (SVM):", scores_svm.mean())

R² em cada fold (SVM): [-0.12888945 -0.13722987 -0.14160945 -0.1174921  -0.13303388]
R² médio (SVM): -0.13165095082583167


In [12]:
from sklearn.ensemble import RandomForestRegressor

# Pipeline para Random Forest sem normalização ou padronização
preprocessor_rf = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), features_nominais),
        ('num', 'passthrough', [col for col in features if col not in features_nominais])
    ]
)

pipeline_rf = Pipeline([
    ('preprocess', preprocessor_rf),
    ('regressor', RandomForestRegressor(random_state=87))
])

# Treinando o pipeline com Random Forest
pipeline_rf.fit(X_train, y_train)

# Avaliando o modelo com cross-validation
scores_rf = cross_val_score(pipeline_rf, X_train, y_train, cv=kf, scoring='r2')
print("R² em cada fold (Random Forest):", scores_rf)
print("R² médio (Random Forest):", scores_rf.mean())

R² em cada fold (Random Forest): [0.93860619 0.94622928 0.94807594 0.94694507 0.92083283]
R² médio (Random Forest): 0.9401378622663878


In [14]:
# Criar uma cópia do dataframe
df_log = df.copy()

# Aplicar log1p para lidar com valores zero
df_log['budget'] = np.log1p(df_log['budget'])
df_log['revenue'] = np.log1p(df_log['revenue'])

# Usar os dados transformados com log
X_log = df_log[features]
y_log = df_log['revenue']

# Dividir em treino e teste
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(X_log, y_log, test_size=0.2, random_state=87)

In [15]:
# Pipeline para colunas nominais com StandardScaler e SVM
preprocessor_standard = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), features_nominais),
        ('num', StandardScaler(), [col for col in features if col not in features_nominais])
    ]
)

pipeline_svm_log = Pipeline([
    ('preprocess', preprocessor_standard),
    ('regressor', SVR())
])

# Treinando o pipeline com SVM nos dados com log
pipeline_svm_log.fit(X_train_log, y_train_log)

# Avaliando o modelo com cross-validation
scores_svm_log = cross_val_score(pipeline_svm_log, X_train_log, y_train_log, cv=kf, scoring='r2')
print("R² em cada fold (SVM com log):", scores_svm_log)
print("R² médio (SVM com log):", scores_svm_log.mean())


R² em cada fold (SVM com log): [0.68275119 0.67084567 0.65140393 0.66794528 0.66284977]
R² médio (SVM com log): 0.6671591682503257


In [13]:
from sklearn.metrics import mean_squared_error, r2_score

# Dicionário de modelos
modelos = {
    'KNN (MaxAbsScaler)': pipeline,
    'KNN (MinMaxScaler)': pipeline_minmax,
    'Random Forest': pipeline_rf,
    'SVM (StandardScaler)': pipeline_svm,
    'SVM (StandardScaler com log)': pipeline_svm_log
}

resultados = []

for nome, modelo in modelos.items():
    y_pred = modelo.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    resultados.append({'Modelo': nome, 'MSE': mse, 'R²': r2})

resultados_df = pd.DataFrame(resultados)
print(resultados_df)

                         Modelo           MSE        R²
0            KNN (MaxAbsScaler)  1.122440e+16  0.667429
1            KNN (MinMaxScaler)  1.136187e+16  0.663355
2                 Random Forest  8.605332e+14  0.974503
3          SVM (StandardScaler)  3.813539e+16 -0.129925
4  SVM (StandardScaler com log)  4.173893e+16 -0.236696


In [16]:
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline

# Selecionar apenas as features numéricas para o K-means
features_numericas = [col for col in features if col not in features_nominais]
X_numerico = df[features_numericas]

# Pipeline com MinMaxScaler e KMeans
pipeline_kmeans = make_pipeline(
    MinMaxScaler(),
    KMeans(n_clusters=5, random_state=87, n_init=10)
)

# Ajustar o pipeline aos dados
pipeline_kmeans.fit(X_numerico)

# Adicionar os rótulos dos clusters ao dataframe
df['cluster_kmeans'] = pipeline_kmeans.named_steps['kmeans'].labels_

# Visualizar a contagem de filmes por cluster
print(df['cluster_kmeans'].value_counts())

cluster_kmeans
2    7648
3    6009
1    5874
0    1484
4    1157
Name: count, dtype: int64


In [17]:
# Análise dos clusters criados pelo KMeans

# Visualizar estatísticas gerais dos clusters
cluster_summary = df.groupby('cluster_kmeans')[features_numericas + ['revenue']].mean()
print("Média das variáveis numéricas por cluster:")
print(cluster_summary)

# Verificar a distribuição dos gêneros por cluster
generos_por_cluster = df.groupby('cluster_kmeans')['genres_names'].value_counts().unstack(fill_value=0)
print("\nDistribuição dos gêneros por cluster:")
print(generos_por_cluster)

# Insights para recomendação:
# - Cada cluster representa grupos de filmes com características semelhantes (ex: orçamento, popularidade, receita, etc).
# - Para recomendar filmes, pode-se sugerir títulos do mesmo cluster de um filme que o usuário gostou, pois compartilham perfis parecidos.
# - A análise dos gêneros mais frequentes em cada cluster pode ajudar a personalizar recomendações de acordo com preferências de gênero.
# - Clusters com maior média de receita ou popularidade podem indicar filmes mais "mainstream", enquanto clusters com valores menores podem indicar nichos ou filmes independentes.

# Exemplo: mostrar os 3 principais gêneros de cada cluster
top_generos = generos_por_cluster.apply(lambda x: x.sort_values(ascending=False).head(3), axis=1)
print("\nTop 3 gêneros por cluster:")
print(top_generos)

Média das variáveis numéricas por cluster:
                popularity        budget     runtime  vote_average  \
cluster_kmeans                                                       
0                14.240410  3.338751e+06  113.133423      6.914766   
1                30.257059  2.642729e+07  108.154069      6.310658   
2                31.179380  2.825503e+07  110.999085      6.465541   
3                35.819016  2.541794e+07  107.478782      6.316056   
4               163.945744  1.486230e+08  126.229905      7.123475   

                  vote_count  release_year  release_month       revenue  
cluster_kmeans                                                           
0                 745.930593   1954.574124       8.039084  1.919574e+07  
1                1317.229656   2003.742935       6.605720  6.757431e+07  
2                1586.071783   2004.891213      10.450968  6.784767e+07  
3                1303.231153   2003.870694       2.539690  5.570160e+07  
4               11452.